# Vasculature simulation use-case

### Connect to back-end

In [1]:
from brayns import Client
from circuitexplorer import CircuitExplorer
from bioexplorer import BioExplorer, TransferFunction, Widgets, MovieMaker

url = 'localhost:5000'

brayns = Client(url)
ce = CircuitExplorer(brayns)
be = BioExplorer(url)
mm = MovieMaker(be)

### Load vasculature

In [2]:
data_folder = '/media/favreau/Documents/medias/ngv/'

In [3]:
vasculature_file = data_folder + 'graph_with_endfeet.h5'
vasculature_model = ce.load_vasculature(
    name='Vasculature', path=vasculature_file,
    use_sdf=False, geometry_quality=ce.GEOMETRY_QUALITY_HIGH
)
vasculature_model_id = vasculature_model['id']

In [4]:
ce.set_vasculature_default_materials(
    model_id=vasculature_model_id, palette_name='Set1_r',
    shading_mode=ce.SHADING_MODE_CARTOON
)

### Set camera default position and rendering parameters

In [5]:
status = brayns.set_camera(
    orientation = [-0.188, 0.28, 0.036, 0.940],
    position = [1577, 2183, 2511], target = [333, 1279, 646])

In [6]:
status = brayns.set_renderer(
    current='circuit_explorer_advanced',
    subsampling=4, max_accum_frames=64)
params = brayns.CircuitExplorerAdvancedRendererParams()
params.epsilon_factor = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = brayns.set_renderer_params(params)

## Simulation report

### Simulation transfer function

In [7]:
tf = TransferFunction(
    bioexplorer=BioExplorer(url), name='Reds',
    model_id=vasculature_model['id'], size=16, alpha=1.0
)

### Set materials to display simulation data

In [8]:
ce.set_vasculature_default_materials(
    model_id=vasculature_model_id, palette_name='Reds',
    map_simulation_data=True)

### Attach simulation report to the vasculature

In [9]:
''' Report file names and value ranges '''
reports = ['report_flows', 'report_pressures', 'report_radii']
reports_ranges = [[-0.5, 1.5], [0, 0.0015], [0, 45]]

''' Attach report to vasculature '''
report_id = 0

report_filename = data_folder + reports[report_id] +'.h5'
ce.attach_vasculature_report(
    model_id=vasculature_model_id,
    path=report_filename, debug=False)
tf.set_range(reports_ranges[report_id])

### Widget example

In [10]:
from ipywidgets import Select, HBox

def update_report(value):
    ce.attach_vasculature_report(
        model_id=vasculature_model_id,
        path=data_folder + reports[value.new] + '.h5')
    tf.set_palette(ce.COLOR_MAPS[colormap_selector.index])
    tf.set_range(reports_ranges[report_selector.index])
    brayns.set_renderer()
    
def update_colormap(value):
    tf.set_palette(ce.COLOR_MAPS[colormap_selector.index])
    tf.set_range(reports_ranges[report_selector.index])
    brayns.set_renderer()

report_selector = Select(options=reports)
report_selector.observe(update_report, 'index')
colormap_selector = Select(options=ce.COLOR_MAPS)
colormap_selector.observe(update_colormap, 'index')
hbox = HBox([report_selector, colormap_selector])
display(hbox)

## Snapshots

### Single snapshot

In [11]:
''' Create image and save it to disk '''
mm.create_snapshot(
    size=[1080, 1920], samples_per_pixel=64,
    base_name='vasculature_2k_v1', path='/tmp')

' Create image and save it to disk '

### Simple movie

In [12]:
for frame in range(720):
    ''' Apply geometry modifications to the vasculature '''
    ce.apply_vasculature_geometry_report(
        path=data_folder + 'report_radii.h5',
        model_id=vasculature_model_id,
        frame=frame, amplitude=0.75, debug=True
    )
    ''' Set animation frame (for the mapping of the simulation report) '''
    brayns.set_animation_parameters(current=frame)

    ''' Create image and save it to disk '''
    mm.create_snapshot(
        size=[1080, 1080], samples_per_pixel=32,
        base_name='%05d' % frame, path='/tmp')